## GROUP 함수
### 1. AGGREGATION FUNCTION
- COUNT, SUM, AVG, MAX, MIN의 각종 집계 함수 포함

---
### 2. GROUP FUCTION

#### 2-1 ROLLUP 
- 소그룹간의 소계를 계산하는 함수
- GROUP BY의 확장된 형태로 사용하기가 쉬우며 병렬로 수행이 가능하기 때문에 매우 효과적이다.
- 시간 및 지역처럼 계층적 분류를 포함하고 있는 데이터의 집계에 적합하도록 되어있다.

ROLLUP 에 지정된 GROUPING COLUMNS의 LIST는 SUBTOTAL을 생성하기 위해 사용되어진다.   
GROUPING COLUMNS의 수를 N이라고 했을 때 N+1 LEVEL의 SUBTOTAL이 생성된다.   
ROLLUP의 인수는 계층 구조이므로 인수 순서가 바뀌면 수행 결과도 바뀌게 되므로 인수의 순서에 주의해야 한다.   

In [ ]:
# 부서명과 업무명을 기준으로 집계한 일반적인 GROUP BY 문장에 ROLLUP 함수를 사용한다.
SELECT DNAME, JOB,
        COUNT(*) "TOTAL EMPL",
        SUM(SAL) "TOTAL SAL"
FROM EMP, DEPT
WHERE EDPT.DEPTNO = EMP.DEPTNO
GROUP BY ROLLUP (DNAME, JOB);

위 SQL문의 수행 결과 일반적으로 GROUP BY한 결과와 달리 2개의 GROUPING COLUMNS (DNAME, JOB)에 대하여 추가 LEVEL의 집계가 생성된다.

> L1. GROUP BY 수행시 생성되는 표준 집계 (ROLLUP 활용하지 않은 결과와 동일)     
> L2. DNAME 별 모든 JOB의 SUBTOTAL (추가)   
> L3. GRAND TOTAL 

GROUP BY 수행시 생성되는 표준 집계에는 별도의 정렬을 지원하지 않으므로 정렬을 위해서는 별도의 ORDER BY절을 사용해야 한다.

---
##### GROUPING 함수 사용
- ROLLUP이나  CUBE에 의한 소계가 계산된 결과에는 GROUPING(EXPR) = 1이 표시되고 그 외의 결과에는 '0'이 표시된다.

In [ ]:
SELECT DNAME, GROUPING(DNAME), JOB, GROUPING(JOB), COUNT(*) 'TOTAL EMPL', SUM(SAL) 'TOTAL SAL'
FROM EMP, DEPT
WHERE DEPT.DEPTNO = EMP.DEPTNO
GROUP BY ROLLUP (DNAME, JOB)

GROUPING(DNAME) 과 GROUPING(JOB) COLUMNS 값으로 소계가 계산된 행은 '1' 그렇지 않은 행은 '0'이 표시된다.   
    → DNAME별 JOB의 SUB TOTAL : GROUPING(DNAME) = 0 GROUPING(JOB) = 1   
    → GRAND TOTAL : GROUPING(DNAME) = 1, GROUPING(JOB) = 1

---
##### GROUPING 함수 + CASE 사용
ROLLUP 함수를 추가한 집계 보고서에서 집계 레코드를 구분할 수 있는 GROUPING 함수와 CASE 함수를 함께 사용한 SQL문장을 작성한다.   
> → DNAME과 JOB COLUMNS에서 SUBTOAL이 출력되는 행에서 사용자 정의 문자열인 'ALL ＊'이 출력된다.

In [ ]:
SELECT
CASE GROUPING(DNAME) WHEN 1 THEN 'ALL DEPARTMENTS' ELSE DNAME END AS DNAME,
CASE GROUPING(JOB) WHEN 1 THEN 'ALL JOBS' ELSE JOB END AS JOB,
COUNT(*) 'TOTAL EMPL',
SUM(SAL) 'TOTAL SAL'
FROM EMP, DEPT
WHERE DEPT.DEPTNO = DMP.DEPTNO
GROUP BY ROLLUP (DNAME,JOB);

# ORACLE의 경우에는 DECODE 함수를 사용해서 좀 더 짧게 표현할 수 있다.
SELECT
DECODE(GROUPING(DNAME), 1, 'ALL DEPARTMENTS', DNAME) AS DNAME)
DECODE(GROUPING(JOB),1,'ALL JOBS', JOB) AS JOB)
COUNT(*) 'TOTAL EMPL',
SUM(SAL) 'TOTAL SAL'
FROM EMP, DEPT
WHERE DEPT.DEPTNO = DMP.DEPTNO
GROUP BY ROLLUP (DNAME,JOB);

##### ROLLUP 함수 일부 사용
GROUP BY ROLLUP(DNAME,JOB) → GROUP BY DNAME ROLLUP(JOB)

In [ ]:
SELECT
CASE GROUPING(DNAME) WHEN 1 THEN 'ALL DEPARTMENTS' ELSE DNAME END AS DNAME,
CASE GROUPING(JOB) WHEN 1 THEN 'ALL JOBS' ELSE JOB END AS JOB,
COUNT(*) 'TOTAL EMPL',
SUM(SAL) 'TOTAL SAL'
FROM EMP, DEPT
WHERE DEPT.DEPTNO = DMP.DEPTNO
GROUP BY DNAME ROLLUP (JOB);
#  실행 결과 : 기존의 SQL문과 달리 마지막 GRAND TOTAL 줄이 출력되지 않음

##### ROLLUP함수 결합 칼럼 사용
 JOB과 MGR을 하나의 집합으로 간주하여 ROLLUP 결과를 출력한다. → 괄호 내 각 칼럼별 집계를 구하지 않고 GRAND TOTAL에서만 활용된다.

In [ ]:
SELECT DNAME, JOB, MGR,SUM(SAL) 'TOTAL SAL'
FROM CMP, DEPT
WHERE DEPT.DEPTNO = EMP.DEPTNO
GROUP BY ROLLUP (DNAME, (JOB,MGR));

---
#### 2-2 CUBE 
- GROUP BY 항목들 간 다차원적인 소계를 계산할 수 있는 함수
- 결합 가능한 모든 값에 대하여 다차원적인 집계를 생성하여 ROLL UP 에 비해 다양한 데이터를 얻는 장점이 있다.
- 시스템에 부하를 많이 주는 단점이 있다.
- 계층 구조인 ROLLUP과는 달리 평등한 관계이므로 인수의 순서가 바뀌는 경우 행간에 정렬 순서는 바뀔 수 있어도 데이터 결과는 같다.
- 결과에 대한 정렬이 필요한 경우 ORDER BY절에 명시적으로 정렬 칼럼이 표시가 되어야 한다.

> → CUBE는 GROUPING COLUMNS의 수가 N이라면 2^N LEVEL의 SUBTOTAL을 리턴한다.

In [ ]:
SELECT
CASE GROUPING(DNAME) WHEN 1 THEN 'ALL DEPARTMENTS' ELSE DNAME END AS DNAME,
CASE GROUPING(JOB) WHEN 1 THEN 'ALL JOBS' ELSE JOB END AS JOB,
COUNT(*) 'TOTAL EMPL',
SUM(SAL) 'TOTAL SAL'
FROM EMP, DEPT
WHERE DEPT.DEPTNO = DMP.DEPTNO
GROUP BY CUBE (DNAME,JOB);

# 위의 실행 결과는 모든 DEPERTMENTS에 분포된 동일한 JOBS에 대한 SUB TOTAL까지도 포함한다. 
# !ROLLUP 사용시 JOBS는 DEPERTMENT의 계층의 하위 항목이므로 해달 결과는 리턴하지 않았다!

In [ ]:
# 위 SQL의 실행 결과와 동일한 결과를 리턴하기 위해 UNION ALL을 사용해볼수도 있다. 
# 가독성 떨어지며 같은 테이블에 4번이나 접근해야하는 단점! CUBE사용의 우월성 잘 보여줌!
SELECT DNAME, JOB, COUNT(*) "TOTAL EMPL", SUM(SAL) "TOTAL SAL"
FROM EMP, DEPT
WHERE EDPT.DEPTNO = EMP.DEPTNO
GROUP BY DNAME, JOB
UNION ALL
SELECT DNAME, JOB, COUNT(*) "TOTAL EMPL", SUM(SAL) "TOTAL SAL"
FROM EMP, DEPT
WHERE EDPT.DEPTNO = EMP.DEPTNO
GROUP BY DNAME
UNION ALL
SELECT DNAME, JOB, COUNT(*) "TOTAL EMPL", SUM(SAL) "TOTAL SAL"
FROM EMP, DEPT
WHERE EDPT.DEPTNO = EMP.DEPTNO
GROUP BY JOB
UNION ALL
SELECT DNAME, JOB, COUNT(*) "TOTAL EMPL", SUM(SAL) "TOTAL SAL"
FROM EMP, DEPT
WHERE EDPT.DEPTNO = EMP.DEPTNO;

#### 2-3 GROUPING SETS 
- 특정 항목에 대한 소계를 계산하는 함수
- 원하는 부분의 소계만 손쉽게 추출할 수 있는 장점이 있다.
- 인수들에 대한 개별 집계를 구할 수 있으며, 이때 표시된 인수들 간에는 평등한 관계로 인수의 순서가 바뀌어도 결과는 같다.
- 결과에 대한 정렬이 필요한 경우는 ORDER BY 절에 명시적으로 정렬 칼럼이 표시되어야 한다.

In [ ]:
SELECT
CASE GROUPING(DNAME) WHEN 1 THEN 'ALL DEPARTMENTS' ELSE DNAME END AS DNAME,
CASE GROUPING(JOB) WHEN 1 THEN 'ALL JOBS' ELSE JOB END AS JOB,
COUNT(*) 'TOTAL EMPL',
SUM(SAL) 'TOTAL SAL'
FROM EMP, DEPT
WHERE DEPT.DEPTNO = DMP.DEPTNO
GROUP BY GROUPING SETS (DNAME,JOB);

# SQL문의 실행 결과 '소계'를 나타내는 행만 출력되며 개별 값은 출력 되지 않는다. → CUBE의 결과에서 개별 값 행 삭제한것과 동일한 결과.

In [ ]:
SELECT DNAME, JOB, MGR,SUM(SAL) 'TOTAL SAL'
FROM CMP, DEPT
WHERE DEPT.DEPTNO = EMP.DEPTNO
GROUP BY GROUPING SETS ((DNAME,JOB,MGR),(DNAME,JOB),(JOB,MGR));

# GROUPING SETS함수 사용시 괄호로 묶은 집합별로 집계를 구할 수 있다.